# Lomas Client Side: Using Smartnoise-SQL

This notebook showcases how researcher could use lomas platform with Smartnoise-SQL. It explains the different functionnalities provided by the `lomas-client` client library to interact with lomas server.

The secure data are never visible by researchers. They can only access to differentially private responses via queries to the server.

Each user has access to one or multiple projects and for each dataset has a limited budget $\epsilon$, $\delta$.

In this notebook the researcher is a penguin researcher named Dr. Antarctica. She aims to do a grounbdbreaking research on various penguins data.

## Step 1: Install the library
To interact with the secure server on which the data is stored, Dr.Antartica first needs to install the library `lomas-client` on her local developping environment. 

It can be installed via the pip command:

In [ ]:
# !pip install lomas_client

Or using a local version of the client

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))

In [ ]:
from lomas_client.client import Client
import numpy as np

## Step 2: Initialise the client

Once the library is installed, a Client object must be created. It is responsible for sending sending requests to the server and processing responses in the local environment. It enables a seamless interaction with the server. 

The client needs a few parameters to be created. Usually, these would be set in the environment by the system administrator (queen Icebergina) and be transparent to lomas users. In this instance, the following code snippet sets a few of these parameters that are specific to this notebook. 

She will only be able to query on the real dataset if the administrator has previously made her an account in the database, given her access to the PENGUIN dataset and has given her some $\epsilon$, $\delta$ privacy loss budget.

In [ ]:
# The following would usually be set in the environment by a system administrator
# and be tranparent to lomas users.
# Uncomment them if you are running against a Kubernetes deployment.
# They have already been set for you if you are running locally within a devenv or the Jupyter lab set up by Docker compose.

import os
# os.environ["LOMAS_CLIENT_APP_URL"] = "https://lomas.example.com:443"
# os.environ["LOMAS_CLIENT_KEYCLOAK_URL"] = "https://keycloak.example.com:443"
# os.environ["LOMAS_CLIENT_TELEMETRY__ENABLED"] = "false"
# os.environ["LOMAS_CLIENT_TELEMETRY__COLLECTOR_ENDPOINT"] = "http://otel.example.com:445"
# os.environ["LOMAS_CLIENT_TELEMETRY__COLLECTOR_INSECURE"] = "true"
# os.environ["LOMAS_CLIENT_TELEMETRY__SERVICE_ID"] = "my-app-client"
# os.environ["LOMAS_CLIENT_REALM"] = "lomas"

# We set these ones because they are specific to this notebook.

USER_NAME = "Dr.Antartica"
os.environ["LOMAS_CLIENT_CLIENT_ID"] = USER_NAME
os.environ["LOMAS_CLIENT_CLIENT_SECRET"] = USER_NAME.lower()
os.environ["LOMAS_CLIENT_DATASET_NAME"] = "PENGUIN"

# Note that all client settings can also be passed as keyword arguments to the Client constructor.
# eg. client = Client(client_id = "Dr.Antartica") takes precedence over setting the "LOMAS_CLIENT_CLIENT_ID"
# environment variable.

In [ ]:
client = Client()

And that's it for the preparation. She is now ready to use the various functionnalities offered by `lomas-client`.

## Step 3: Getting dataset metadata

In [ ]:
metadata = client.get_dataset_metadata()
metadata

{'max_ids': 1,
 'rows': 344,
 'row_privacy': True,
 'censor_dims': False,
 'columns': {'species': {'private_id': False,
   'nullable': False,
   'max_partition_length': None,
   'max_influenced_partitions': None,
   'max_partition_contributions': None,
   'type': 'string',
   'cardinality': 3,
   'categories': ['Adelie', 'Chinstrap', 'Gentoo']},
  'island': {'private_id': False,
   'nullable': False,
   'max_partition_length': None,
   'max_influenced_partitions': None,
   'max_partition_contributions': None,
   'type': 'string',
   'cardinality': 3,
   'categories': ['Torgersen', 'Biscoe', 'Dream']},
  'bill_length_mm': {'private_id': False,
   'nullable': False,
   'max_partition_length': None,
   'max_influenced_partitions': None,
   'max_partition_contributions': None,
   'type': 'float',
   'precision': 64,
   'lower': 30.0,
   'upper': 65.0},
  'bill_depth_mm': {'private_id': False,
   'nullable': False,
   'max_partition_length': None,
   'max_influenced_partitions': None,
   'm

In [ ]:
nb_penguin = metadata['rows']
print(f"Number of penguins: {nb_penguin}.")

Number of penguins: 344.


In [ ]:
columns = metadata["columns"].keys()
columns

dict_keys(['species', 'island', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g', 'sex'])

## Step 4: Average bill length with Smartnoise-SQL

### Query dummy dataset

Now that she has an idea of what the data looks like, she wants to start querying the real dataset to for her research. However, before this, other tools are at her disposal to reduce potential error risks and avoid spending budget on irrelevant queries. Of course, this does not have any impact on the budget.

It is possible to specify the flag `dummy=True` in the various queries to perform the query on the dummy dataset instead of the real dataset and ensure that the queries are doing what is expected of them. 

In [ ]:
# Average bill length in mm
QUERY = "SELECT AVG(bill_length_mm) AS avg_bill_length_mm FROM df"

In [ ]:
EPSILON = 0.5
DELTA = 1e-5

In [ ]:
# On the remote server dummy dataframe
dummy_res = client.smartnoise_sql.query(
    query = QUERY,  
    epsilon = EPSILON,
    delta = DELTA,
    dummy = True,
)

In [ ]:
avg_bl_dummy = np.round(dummy_res.result.df["avg_bill_length_mm"][0], 2)
f"Average bill length on dummy: {avg_bl_dummy}mm."

'Average bill length on dummy: 48.8mm.'

### Estimate cost of a query
Dr. Antartica checks the budget that computing the average bill length will really cost her if she asks the query with an `epsilon` and a `delta`.

In [ ]:
cost = client.smartnoise_sql.cost(
    query = QUERY, 
    epsilon = EPSILON, 
    delta = DELTA,
)

In [ ]:
f'This query would actually cost her {cost.epsilon} epsilon and {cost.delta} delta.'

'This query would actually cost her 1.0 epsilon and 5.000000000032756e-06 delta.'

This is actually twice as much as what she initially put in. In the background, Smartnoise-SQL decomposes the DP query in multiple other queries and the budget given as input is spent on each of these sub-queries. Here for the average, we need a sum divided by a count, hence `EPSILON` is spent once for the sum and then once more for the count. (see NOTE below for tips and explanation).

### Overide DP mechanism

She wants to use another DP-mechanism for this query. She can change it via the `mechanism` argument. See Smartnoise-SQL documentation [here for overriding mechanisms](https://docs.smartnoise.org/sql/advanced.html#overriding-mechanisms).

In [ ]:
# On the remote server dummy dataframe
dummy_res = client.smartnoise_sql.query(
    query = QUERY,  
    epsilon = EPSILON,
    delta = DELTA,
    mechanisms = {"count": "gaussian", "sum_float": "laplace"},
    dummy = True,
)

In [ ]:
avg_bl_dummy = np.round(dummy_res.result.df["avg_bill_length_mm"][0], 2)
f"Average bill length on dummy: {avg_bl_dummy}mm."

'Average bill length on dummy: 55.31mm.'

In [ ]:
cost = client.smartnoise_sql.cost(
    query = QUERY, 
    epsilon = EPSILON, 
    delta = DELTA,
    mechanisms = {"count": "gaussian", "sum_float": "laplace"}
)
cost

CostResponse(epsilon=1.0, delta=1.4999949999983109e-05)

### Query real dataset
Dr. Antartica is ready to query the real dataset and get a differentially private response for the average bill length. The `dummy` flag is False by default, so setting it is optional. She uses the values of `epsilon` and `delta` that she selected just before.

Careful: This command DOES spend the budget of the user and the remaining budget is updated for every query.

In [ ]:
avg_bill_length_response = client.smartnoise_sql.query(
    query = QUERY,  
    epsilon = EPSILON, 
    delta = DELTA,
    mechanisms = {"count": "gaussian", "sum_float": "laplace"},
    dummy = False
)

In [ ]:
avg_bill_length = avg_bill_length_response.result.df['avg_bill_length_mm'].iloc[0]
print(f"Average bill length on private data: {np.round(avg_bill_length, 2)}mm.")

Average bill length on private data: 43.78mm.


After each query on the real dataset, the budget informations are also returned to the researcher. It is possible possible to check the remaining budget again afterwards:

### Postprocess 

It is also possible to use the 'postprocess' argument from Smartnoise-SQL [see its documentation here](https://docs.smartnoise.org/sql/advanced.html#postprocess) by specifying it in the query.

In [ ]:
dummy_res = client.smartnoise_sql.query(
    query = QUERY,  
    epsilon = EPSILON,
    delta = DELTA,
    postprocess = True,
    dummy = True,
)
dummy_res

QueryResponse(epsilon=1.0, delta=5.000000000032756e-06, requested_by='Dr.Antartica', result=SmartnoiseSQLQueryResult(res_type=<DPLibraries.SMARTNOISE_SQL: 'smartnoise_sql'>, df=   avg_bill_length_mm
0           48.370249))

In [ ]:
dummy_res = client.smartnoise_sql.query(
    query = QUERY,  
    epsilon = EPSILON,
    delta = DELTA,
    postprocess = False,
    dummy = True,
)
dummy_res

QueryResponse(epsilon=1.0, delta=5.000000000032756e-06, requested_by='Dr.Antartica', result=SmartnoiseSQLQueryResult(res_type=<DPLibraries.SMARTNOISE_SQL: 'smartnoise_sql'>, df=         res_0       res_1
0  4636.171538  100.761072))

## Step 4: Penguin statistics

### Confidence intervals for flipper length over the whole population

She is first interested to have a better idea of the distribution of bill length of all species. She already has the number of penguins (=number of rows as `max_ids=1`) from the metadata and the average bill length from step 3, so she just needs to compute the standard deviation. As it is just an exploration step, she uses very little budget values.

In [ ]:
QUERY = "SELECT STD(bill_length_mm) AS std_bill_length_mm FROM df"

She again first verifies that her query works on the dummy dataset:

In [ ]:
dummy_res = client.smartnoise_sql.query(
    query = QUERY, 
    epsilon = 0.5, 
    delta = 1e-5, 
    dummy = True
)

In [ ]:
dummy_std = np.round(dummy_res.result.df['std_bill_length_mm'].iloc[0], 2)
f"The dummy standard variation is {dummy_std}."

'The dummy standard variation is 19.0.'

The syntax of the query works, now she checks the budget:

In [ ]:
cost = client.smartnoise_sql.cost(
    query = QUERY, 
    epsilon = 0.5, 
    delta = 1e-5
)

In [ ]:
f'This query would actually cost her {cost.epsilon} epsilon and {cost.delta} delta.'

'This query would actually cost her 1.5 epsilon and 5.000000000032756e-06 delta.'

This time it is three times the budget because the standard deviation needs the average, then a difference and a count again. 

In [ ]:
response = client.smartnoise_sql.query(
    query = QUERY,
    epsilon = 0.5,
    delta = 1e-5
)

In [ ]:
std_bill_length = response.result.df['std_bill_length_mm'].iloc[0]
print(f"Standard deviation of bill length: {np.round(std_bill_length, 2)}.")

Standard deviation of bill length: 7.12.


She can now do all the postprocessing that she wants with the returned data without increasing the privacy risk. 

In [ ]:
# Get standard error
standard_error = std_bill_length/np.sqrt(nb_penguin)
print(f"Standard error of bill length: {np.round(standard_error, 2)}.")

Standard error of bill length: 0.38.


In [ ]:
 # Compute the 95% confidence interval
ZSCORE = 1.96
lower_bound, upper_bound = avg_bill_length - ZSCORE*standard_error, avg_bill_length + ZSCORE*standard_error
print(f"The 95% confidence interval of the bill length of all penguins is [{np.round(lower_bound, 2)}, {np.round(upper_bound, 2)}].")

The 95% confidence interval of the bill length of all penguins is [43.03, 44.54].


## Note on budget with Smartnoise-SQL (Advanced)

All of these queries will cost the same budget in Smartnoise-SQL. The reason is that the smartnoise-sql translates the input query in sub queries, finds the answer for each sub query for the budget in input and then assembles the results. For the first 'standard deviation' query, it requires a count, an average, and only then the computation for the standard deviation. Hence, to save budget it is better to make a general query directly and retrieve all the 'sub-answers'.

In [ ]:
epsilon = 1.0
delta = 1e-5

In [ ]:
QUERY = "SELECT STD(bill_length_mm) AS std_bill_length_mm FROM df"
cost = client.smartnoise_sql.cost(query = QUERY, epsilon = epsilon, delta = delta)
cost

CostResponse(epsilon=3.0, delta=5.000000000032756e-06)

In [ ]:
QUERY = "SELECT AVG(bill_length_mm) AS avg_bl, STD(bill_length_mm) as std_bl FROM df"
cost = client.smartnoise_sql.cost(query = QUERY, epsilon = epsilon, delta = delta)
cost

CostResponse(epsilon=3.0, delta=5.000000000032756e-06)

In [ ]:
QUERY = "SELECT COUNT(bill_length_mm) AS count_bl, AVG(bill_length_mm) AS avg_bl, STD(bill_length_mm) as std_bl FROM df"
cost = client.smartnoise_sql.cost(query = QUERY, epsilon = epsilon, delta = delta)
cost

CostResponse(epsilon=3.0, delta=5.000000000032756e-06)

A way to know the sub-queries of a query is to use the following Smartnoise-SQL code:

In [ ]:
# Convert metadata to Smartnoise-SQL compliant metadata
metadata = dict(metadata)
metadata.update(metadata["columns"])
del metadata["columns"]
snsql_metadata = {"": {"": {"df": metadata}}}

In [ ]:
# Write the query to inspect
QUERY = "SELECT STD(bill_length_mm) as std_bl FROM df"
#QUERY = "SELECT COUNT(*) as nb_row FROM df"

In [ ]:
from snsql.sql.private_rewriter import Rewriter
rewriter = Rewriter(snsql_metadata)
rewriter.options.row_privacy = metadata["row_privacy"]
rewriter.options.max_contrib = metadata["max_ids"]
dp_query = rewriter.query(QUERY)
dp_query

The original dp query is represented as one query:

In [ ]:
dp_query._named_symbols

{'std_bl': <snsql._ast.tokens.Symbol at 0x76b1d81e9010>}

But has 4 named symbols inside: 2 alias for the 2 SQL subqueries 
- 'keycount' for 'count_bill_length_mm',
- 'sum_alias_0xxxx' for 'sum_bill_length_mm'

In [ ]:
subquery = dp_query.source.relations[0].primary.query
syms = subquery._named_symbols
syms

{'keycount': <snsql._ast.tokens.Symbol at 0x76b1d81c5ee0>,
 'sum_alias_0xf3d7': <snsql._ast.tokens.Symbol at 0x76b1d81c5b20>,
 'count_bill_length_mm': <snsql._ast.tokens.Symbol at 0x76b1d81c5eb0>,
 'sum_bill_length_mm': <snsql._ast.tokens.Symbol at 0x76b1d8142ba0>}

This last query with `group_by` will cost the same because `max_ids=1` (a penguin appears in the dataset at most once) and so the `group_by` is applied on different partitions of the population.

In [ ]:
QUERY = "SELECT COUNT(bill_length_mm) AS count_bl, AVG(bill_length_mm) AS avg_bl, STD(bill_length_mm) as std_bl FROM df GROUP BY species"
cost = client.smartnoise_sql.cost(query = QUERY, epsilon = epsilon, delta = delta)
cost

CostResponse(epsilon=3.0, delta=5.000000000032756e-06)

NOTE: in the current code of Smartnoise-SQL, there is no odometer. Meaning all queries are independant. If someone first queries the private dataset for a count, then a second time for the average and then for the standard deviation then the total cost will be added: 3 count + 2 average + 1 std. That's why it is better to do everything in one query.

### Hypothesis testing

So, Dr. Antartica has now the opportunity to study the various penguins dimensions and will do an hypotheses testing analysis to discover if flipper length differ between the penguins species.

She will do a two-tailed two-sample $t$ test.

- The null hypothese $H_0$ is flipper length does not differ between species.
- The alternative hypothesis $H_a$ is flipper length does differ between species.

She set the level of significance at 0.05.

In [ ]:
CRITICAL_VALUE = 0.05

In [ ]:
QUERY = "SELECT \
        species AS species, \
        COUNT(bill_length_mm) AS nb_penguin,  \
        AVG(bill_length_mm) AS avg_bill_length_mm, \
        STD(bill_length_mm) AS std_bill_length_mm \
        FROM df GROUP BY species"

She checks the remaining budget:

In [ ]:
client.get_remaining_budget()

RemainingBudgetResponse(remaining_epsilon=7.5, remaining_delta=0.004980000049999984)

She estimates how much budget it would really cost:

In [ ]:
client.smartnoise_sql.cost(query = QUERY, epsilon = 1.0, delta = 1e-4)

CostResponse(epsilon=3.0, delta=4.999999999999449e-05)

The real cost seems to be 3 times the epsilon that she sets. It is a lot but she tries on the dummy dataset to verify all is working properly.

In [ ]:
dummy_res = client.smartnoise_sql.query(query = QUERY, epsilon = 0.1, delta = 1e-8, dummy = True)
dummy_res

InvalidQueryException: SQL Reader generated NAN results. Epsilon: 0.1 and Delta: 1e-08 are too small to generate output.

She did not give enough budget for the query to work. This is why there are 'NANs' in the output. She has to spend more budget for the query to work.

In [ ]:
dummy_res = client.smartnoise_sql.query(query = QUERY, epsilon = 7.5/3, delta = 1e-4, dummy = True)

dummy_res

QueryResponse(epsilon=7.5, delta=4.999999999999449e-05, requested_by='Dr.Antartica', result=SmartnoiseSQLQueryResult(res_type=<DPLibraries.SMARTNOISE_SQL: 'smartnoise_sql'>, df=     species  nb_penguin  avg_bill_length_mm  std_bill_length_mm
0     Adelie          26           49.350321           12.017164
1  Chinstrap          32           48.275682           11.460034
2     Gentoo          39           46.619363           11.623646))

If it errors, she might need to re-run the query a few times until it works. The budget is not affected by dummy queries anyway.

In [ ]:
from lomas_core.error_handler import InvalidQueryException

flipper_length_response = None

for i in range(0, 10):
    try:
        flipper_length_response = client.smartnoise_sql.query(query = QUERY, epsilon = 7.5/3, delta = 1e-4)
        break
    except InvalidQueryException:
        print(f"Attempt number {i} failed.")
        continue

if flipper_length_response is None:
    print("10 unsuccessfull query attempts.")

And now she should not have any remaining budget:

But she can do her post-processing and hypothesis analysis.

In [ ]:
df_flipper = flipper_length_response.result.df
df_flipper

,species,nb_penguin,avg_bill_length_mm,std_bill_length_mm
0,Adelie,150,38.846807,2.482983
1,Chinstrap,66,48.651670,6.125182
2,Gentoo,123,47.478053,2.275091


And finally the $t$-test:

In [ ]:
def t_test(avg_1, avg_2, std_1, std_2, nb_1, nb_2):
    standard_error = (std_1 * (nb_1 - 1) + std_2 * (nb_2 - 1))/(nb_1 + nb_2 - 2)
    return (avg_1 - avg_2)/np.sqrt(standard_error**2*(1/nb_1 + 1 /nb_2))

In [ ]:
nb_0, avg_0, std_0 = df_flipper[['nb_penguin', 'avg_bill_length_mm', 'std_bill_length_mm']].iloc[0]
nb_1, avg_1, std_1 = df_flipper[['nb_penguin', 'avg_bill_length_mm', 'std_bill_length_mm']].iloc[1]
nb_2, avg_2, std_2 = df_flipper[['nb_penguin', 'avg_bill_length_mm', 'std_bill_length_mm']].iloc[2]

In [ ]:
t_01 = t_test(avg_0, avg_1, std_0, std_1, nb_0, nb_1)
t_02 = t_test(avg_0, avg_2, std_0, std_2, nb_0, nb_2)
t_12 = t_test(avg_1, avg_2, std_1, std_2, nb_1, nb_2)

print(f"T test between species 0 and specie 1: {np.round(t_01, 2)}.  Reject null hypothesis: {abs(t_01) > CRITICAL_VALUE}.")
print(f"T test between species 0 and specie 2: {np.round(t_02, 2)}. Reject null hypothesis: {abs(t_02) > CRITICAL_VALUE}.")
print(f"T test between species 1 and specie 2: {np.round(t_12, 2)}.   Reject null hypothesis: {abs(t_12) > CRITICAL_VALUE}.")

T test between species 0 and specie 1: -18.49.  Reject null hypothesis: True.
T test between species 0 and specie 2: -29.7. Reject null hypothesis: True.
T test between species 1 and specie 2: 2.13.   Reject null hypothesis: True.


All t-tests are above the critical value of 0.5. She can reject the null hypothesis.

She finally computes the confidence intervals for the flipper length of each species

In [ ]:
ZSCORE = 1.96
df_flipper['standard_error'] = df_flipper['std_bill_length_mm']/np.sqrt(df_flipper['nb_penguin'])
df_flipper['ci_95_lower_bound'] = df_flipper['avg_bill_length_mm'] - ZSCORE * df_flipper['standard_error']
df_flipper['ci_95_upper_bound'] = df_flipper['avg_bill_length_mm'] + ZSCORE * df_flipper['standard_error']
df_flipper

,species,nb_penguin,avg_bill_length_mm,std_bill_length_mm,standard_error,ci_95_lower_bound,ci_95_upper_bound
0,Adelie,150,38.846807,2.482983,0.202735,38.449447,39.244167
1,Chinstrap,66,48.651670,6.125182,0.753958,47.173913,50.129428
2,Gentoo,123,47.478053,2.275091,0.205138,47.075982,47.880124
